# Upload the dataset

# Reading dataset

In [ ]:
import pandas as pd

movies = pd.read_csv('movies.csv')
movies.head(2)

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 4809 non-null   object 
 1   original_title        4809 non-null   object 
 2   tagline               3965 non-null   object 
 3   overview              4806 non-null   object 
 4   genres                4809 non-null   object 
 5   cast                  4809 non-null   object 
 6   director              4779 non-null   object 
 7   keywords              4809 non-null   object 
 8   original_language     4809 non-null   object 
 9   popularity            4809 non-null   float64
 10  production_companies  4809 non-null   object 
 11  production_countries  4809 non-null   object 
 12  release_date          4808 non-null   object 
 13  revenue               4809 non-null   int64  
 14  runtime               4807 non-null   float64
 15  spoken_languages     

# Dropping not required columns

In [ ]:
required_columns =['title','original_title', 'tagline', 'keywords', 'overview', 'genres', 'cast', 'director']
movies = movies[required_columns]

In [ ]:
movies.isna().sum()

title               0
original_title      0
tagline           844
keywords            0
overview            3
genres              0
cast                0
director           30
dtype: int64

In [ ]:
movies.fillna(' ', inplace=True)
movies.isna().sum()

title             0
original_title    0
tagline           0
keywords          0
overview          0
genres            0
cast              0
director          0
dtype: int64

In [ ]:
movies.iloc[0]

title                                                        Avatar
original_title                                               Avatar
tagline                                 Enter the World of Pandora.
keywords          ['culture clash', 'future', 'space war', 'spac...
overview          In the 22nd century, a paraplegic Marine is di...
genres            ['Action', 'Adventure', 'Fantasy', 'Science Fi...
cast              ['Sam Worthington', 'Zoe Saldana', 'Sigourney ...
director                                              James Cameron
Name: 0, dtype: object

# Create movie CONTENT

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['director'] = movies['director'].apply(lambda x: x.replace(" ","") )

## Converting List to Strings

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x: ','.join(map(str, x)))
movies['keywords'] = movies['keywords'].apply(lambda x: ','.join(map(str, x)))
movies['cast'] = movies['cast'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
movies['content'] = movies['title'] + ' ' + movies['overview'] + ' ' + movies['keywords'] + ' ' + movies['cast'] + ' ' + movies ['director']
movies['content']

0       Avatar In the 22nd century, a paraplegic Marin...
1       Pirates of the Caribbean: At World's End Capta...
2       Spectre A cryptic message from Bond’s past sen...
3       The Dark Knight Rises Following the death of D...
4       John Carter John Carter is a war-weary, former...
                              ...                        
4804    El Mariachi El Mariachi just wants to play his...
4805    Newlyweds A newlywed couple's honeymoon is upe...
4806    Signed, Sealed, Delivered "Signed, Sealed, Del...
4807    Shanghai Calling When ambitious New York attor...
4808    My Date with Drew Ever since the second grade ...
Name: content, Length: 4809, dtype: object

# Natural Language Processing

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)
movie_vectors = vectorizer.fit_transform(movies['content'].values) 

# Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(movie_vectors)

In [ ]:
similarity_df = pd.DataFrame(similarity)
similarity_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802,4803,4804,4805,4806,4807,4808
0,1.000000,0.120720,0.054727,0.181850,0.157633,0.169431,0.085322,0.119674,0.038074,0.041976,0.059856,0.124835,0.030252,0.097009,0.091383,0.138101,0.085341,0.079639,0.091711,0.133738,0.090037,0.062529,0.089699,0.068654,0.065944,0.111964,0.214429,0.270058,0.011245,0.171417,0.075153,0.071986,0.132333,0.060288,0.119584,0.109617,0.176813,0.085004,0.103113,0.068666,...,0.107381,0.061106,0.045187,0.046509,0.056639,0.045939,0.065185,0.058425,0.020405,0.121979,0.152649,0.085769,0.024867,0.091521,0.136437,0.059574,0.066321,0.033706,0.035877,0.099176,0.132461,0.135410,0.030727,0.113159,0.054479,0.029395,0.029669,0.059890,0.061162,0.137434,0.058914,0.054707,0.115616,0.085792,0.100569,0.086665,0.050144,0.106024,0.062859,0.047848
1,0.120720,1.000000,0.099872,0.124117,0.216898,0.046506,0.091782,0.235887,0.078293,0.137685,0.127333,0.106837,0.237146,0.149621,0.219803,0.201105,0.174839,0.137342,0.268327,0.232996,0.192888,0.090521,0.190494,0.088833,0.071821,0.124582,0.231614,0.135855,0.104890,0.190082,0.066411,0.113662,0.171761,0.090056,0.110659,0.212955,0.173911,0.141232,0.156721,0.143715,...,0.119625,0.098157,0.047318,0.070987,0.093188,0.046853,0.141048,0.104784,0.068325,0.154538,0.085329,0.046291,0.086622,0.091154,0.217753,0.161084,0.082485,0.062445,0.103759,0.124093,0.210150,0.234326,0.094350,0.167323,0.135678,0.059018,0.102940,0.075767,0.131434,0.236768,0.053258,0.116830,0.112313,0.166890,0.213747,0.102234,0.085246,0.151463,0.137537,0.065207
2,0.054727,0.099872,1.000000,0.073028,0.045837,0.046143,0.034967,0.101910,0.048610,0.047152,0.049427,0.389960,0.039629,0.089307,0.078866,0.097377,0.093032,0.167796,0.088988,0.106811,0.069212,0.051025,0.120624,0.084180,0.027499,0.067228,0.049404,0.105447,0.013511,0.235121,0.054870,0.017735,0.055744,0.041872,0.025214,0.153334,0.122325,0.094632,0.075334,0.060203,...,0.078743,0.060413,0.043266,0.040737,0.148464,0.082823,0.093168,0.044497,0.023230,0.058373,0.032152,0.027017,0.027781,0.052648,0.111098,0.086375,0.045601,0.033183,0.035927,0.045017,0.113709,0.066407,0.047323,0.053414,0.048472,0.082711,0.036185,0.051839,0.042255,0.066012,0.062993,0.013283,0.033992,0.043828,0.104362,0.079478,0.026120,0.088255,0.041129,0.043192
3,0.181850,0.124117,0.073028,1.000000,0.126776,0.082809,0.146062,0.160151,0.103276,0.093932,0.094969,0.121914,0.053928,0.139591,0.114880,0.196758,0.162537,0.107855,0.119002,0.270752,0.123451,0.083295,0.157697,0.102335,0.108779,0.153828,0.128315,0.125018,0.065484,0.198598,0.129796,0.179356,0.129369,0.079085,0.055013,0.130066,0.208011,0.121181,0.154094,0.129518,...,0.068309,0.096618,0.118633,0.081257,0.174665,0.060290,0.108142,0.125764,0.041201,0.167479,0.067859,0.066746,0.036474,0.106684,0.148441,0.085852,0.076874,0.035836,0.014500,0.051038,0.167149,0.121133,0.081147,0.182192,0.076485,0.046036,0.028939,0.073540,0.056472,0.135917,0.026014,0.058746,0.084078,0.108530,0.109721,0.128345,0.082814,0.141403,0.088217,0.128880
4,0.157633,0.216898,0.045837,0.126776,1.000000,0.080019,0.157128,0.254060,0.063199,0.131756,0.160363,0.123707,0.153126,0.091188,0.151604,0.174659,0.141338,0.174551,0.134740,0.241111,0.175647,0.218845,0.105409,0.101539,0.117110,0.159782,0.290831,0.300271,0.181005,0.149675,0.107633,0.143126,0.178919,0.122182,0.068005,0.161358,0.220728,0.124111,0.077594,0.163682,...,0.118678,0.086691,0.109924,0.064024,0.060497,0.030801,0.101190,0.118962,0.043376,0.184496,0.114104,0.090255,0.049184,0.099242,0.210898,0.101411,0.122975,0.049317,0.019676,0.150661,0.203025,0.214430,0.082783,0.166682,0.073896,0.052092,0.028844,0.069655,0.094886,0.141131,0.082457,0.087538,0.105537,0.097895,0.201148,0.151762,0.058291,0.127700,0.119581,0.084958
...,...,...,...,...,...,...,...,...,...,...,...,...,...

# Recommending Movies

In [ ]:
def recommend(movie):
    # find movie index from dataset
    movies_index = movies[movies['title'] == movie].index[0]
    
    # finding cosine similarities of movie
    distances = similarity[movies_index]
    
    # sorting cosine similarities
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(movies.iloc[i[0]].title)

In [ ]:
movies.title.head(10)

0                                      Avatar
1    Pirates of the Caribbean: At World's End
2                                     Spectre
3                       The Dark Knight Rises
4                                 John Carter
5                                Spider-Man 3
6                                     Tangled
7                     Avengers: Age of Ultron
8      Harry Potter and the Half-Blood Prince
9          Batman v Superman: Dawn of Justice
Name: title, dtype: object

In [ ]:
recommend('Harry Potter and the Half-Blood Prince')

Harry Potter and the Goblet of Fire
Harry Potter and the Order of the Phoenix
Harry Potter and the Philosopher's Stone
Da Sweet Blood of Jesus
Harry Potter and the Chamber of Secrets
